In [1]:
import re, time, shlex, subprocess
import os, subprocess
import xml.etree.ElementTree as ET
from pathlib import Path
import shutil
from __future__ import annotations
import numpy as np, csv
import pandas as pd
import rasterio
from rasterio.windows import Window

In [11]:
# ===== Cell 0: GLOBAL CONFIG (keeps your original names) =====

# --- Edit these per run ---
PLOT = "Teruel"
TILE = "T30TXK"
YEAR = "2022"

# Region tag (unchanged logic)
REGION_TAG = {
    "Lousa": "SerraLousa",
    "Mafra": "SerraMafra",
    "Monsanto": "SerraMonsanto",
    "Pombal": "SerraPombal",
    "Sintra": "SerraSintra",
    "VPA": "SerraVPA",
    # Spain examples:
    "Extremadura": "Extremadura",
    "Leon": "Leon",
    "Huesca": "Huesca",
    "Zaragoza": "Zaragoza",
    "Teruel": "Teruel"
}.get(PLOT, PLOT)

# --- Paths (same names you already use) ---
GPT_EXE   = r"C:\Program Files\esa-snap\bin\gpt.exe"
GRAPH_XML = None  # set in the cell that needs it
MEM_GB    = 8
WORK  = Path(r"A:\S2\work") / PLOT / TILE
DELIV = Path(r"A:\S2\Deliverables") / REGION_TAG / TILE
WORK.mkdir(parents=True, exist_ok=True)
DELIV.mkdir(parents=True, exist_ok=True)

TARGET_RES= 25
BAD_DATES = set() 

# For clipping downstream, AOI can be used
AOI = None
# AOI = Path(r"A:\S2\Shapefiles\Monsanto_limite.shp")
# If computing stats and your AOI has multiple features:
PER_FEATURE = False  # False = dissolve all features (one row per variable). True = one row per feature+var.

# ✅ Keep your original input roots:
EXTRACTED_ROOT = Path(r"A:\S2\SAFE") / PLOT
ZIPPED_ROOT    = Path(r"M:\Project BLS\S2\Data\Aragon") / PLOT / YEAR
# (If you switch campaign later, only change the single ZIPPED_ROOT line above)

# --- CRS, resampling, nodata (shared across cells) ---
TARGET_EPSG       = "EPSG:3763" if REGION_TAG.lower().startswith("serra") else "EPSG:25830"
RESAMPLE_NUMERIC = "bilinear"  # numeric/continuous
RESAMPLE_FLAGS   = "near"      # flags / discrete masks
PLATFORM          = "S2"

# --- Orbit preference (None means “don’t filter”) ---
ORBIT = {
    "SerraMonsanto": "R037",
    "SerraSintra":   "R037",
    "SerraMafra":    "R037",
    # "Extremadura": None,
    # "Leon": None,
}.get(REGION_TAG, None)

# --- Naming helpers (same you already call) ---
def fmt_tag(region: str, year: str, dataset: str, tile: str) -> str:
    return f"{region}_{year}0000_{dataset}_{tile}"

def out_name(study_area: str, year: str, platform: str, variable: str, tile: str) -> str:
    # StudyArea_YYYY0000_Platform_Variable_Tile.tif
    return f"{study_area}_{year}0000_{platform}_{variable}_{tile}.tif"

# --- Lightweight shell helpers you already use ---
def run(cmd: str):
    print(">>", cmd)
    p = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if p.stdout.strip(): print(p.stdout.strip())
    if p.returncode != 0:
        if p.stderr.strip(): print(p.stderr.strip())
        raise RuntimeError("Command failed")

def ensure_dir(p: Path):
    p = Path(p)
    (p.parent if p.suffix else p).mkdir(parents=True, exist_ok=True)

# --- Common subfolders you already reference ---

WORK  = Path(r"A:\S2\work") / PLOT / TILE
DELIV = Path(r"A:\S2\Deliverables") / PLOT / TILE
DELIV.mkdir(parents=True, exist_ok=True)

COMPOSITE_DIR = WORK / "COMPOSITE25"
GLCM_DIR      = WORK / "GLCM25"
REF_DIR       = WORK / "REF25"
BIO_DIR       = WORK / "BIO25"
SCL_DIR       = WORK / "SCL25"
for d in (REF_DIR, BIO_DIR, SCL_DIR):
    d.mkdir(parents=True, exist_ok=True)

D_S2TC = DELIV / "S2TC"
D_BIO  = DELIV / "BIO"
D_GLCM = DELIV / "GLCM"
for d in (WORK, DELIV, D_S2TC, D_BIO, D_GLCM):
    ensure_dir(d)

# --- Band maps (same names you already wired into the builder/README) ---
REF_BANDS = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12"]  # SNAP preprocessing
TC_BANDS  = [f"{b}_TC" for b in REF_BANDS] #SNAP TC + GLCM

NODATA = -9999

# GLCM stat order (no 'GLCM' prefix in output filenames)
GLCM_STATS = ["CONTRAST","DISSIMILARITY","HOMOGENEITY","ASM","ENERGY",
              "MAX","ENTROPY","GLCMMEAN","GLCMVARIANCE","GLCMCORRELATION"]

# TC band labels in order (12)
TC_DATASET_ORDER = ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"] #GDAL dataset semantics

# BIO maps
BIO_TC_ORDER = ["LAI","FAPAR","FCOVER","CAB","CWC"]  # 5-band temporal composite
BIO_DT_ORDER = ["LAI","LAI_FLAGS","CAB","CAB_FLAGS","CWC","CWC_FLAGS",
                "FAPAR","FAPAR_FLAGS","FCOVER","FCOVER_FLAGS"]  # per-date (10)

# --- Publisher mode (optional, same idea; rest of code can read this) ---
PUBLISH_MODE = "release"  # "dev" | "release"
def publisher_policy():
    keep = {
        "dev":     {"WORK", "S2TC", "BIO", "GLCM", "DATASET", "QA", "README", "INVENTORY"},
        "release": {"DATASET", "QA", "README", "INVENTORY"},
    }[PUBLISH_MODE]
    return keep

print(f"[CFG] PLOT={PLOT} | TILE={TILE} | YEAR={YEAR} | REGION_TAG={REGION_TAG} | ORBIT={ORBIT} | EPSG={TARGET_EPSG} | MODE={PUBLISH_MODE}")


[CFG] PLOT=Teruel | TILE=T30TXK | YEAR=2022 | REGION_TAG=Teruel | ORBIT=None | EPSG=EPSG:25830 | MODE=release


In [7]:
# === Extract reflectance, biophysical, and scene classification and resample (SAFE or SAFE.zip) ===

# --- CONFIG ---
GRAPH_XML_TEXT = r"""<graph id="S2_SplitRef_Bio_SCL">
  <version>1.0</version>

  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters><file>${inFile}</file></parameters>
  </node>

  <!-- Branch A: Biophysical fast (Nearest/Median @ targetRes) -->
  <node id="Resample_For_Bio">
    <operator>Resample</operator>
    <sources><sourceProduct refid="Read"/></sources>
    <parameters>
      <targetResolution>${targetRes}</targetResolution>
      <upsampling>Nearest</upsampling>
      <downsampling>Median</downsampling>
    </parameters>
  </node>

  <node id="BiophysicalOp">
    <operator>BiophysicalOp</operator>
    <sources>
      <sourceProduct refid="Resample_For_Bio"/>
    </sources>
    <parameters>
      <sensor>${sensor}</sensor>
      <resolution>20</resolution>
      <computeLAI>true</computeLAI>
      <computeFapar>true</computeFapar>
      <computeFcover>true</computeFcover>
      <computeCab>true</computeCab>
      <computeCw>true</computeCw>
    </parameters>
  </node>

  <node id="Write_Bio">
    <operator>Write</operator>
    <sources><sourceProduct refid="BiophysicalOp"/></sources>
    <parameters><file>${outBio}</file></parameters>
  </node>

  <!-- Branch B: Reflectance (Bicubic/Median @ targetRes), subset to B1..B12 except B10 -->
  <node id="Subset_Reflectance">
    <operator>Subset</operator>
    <sources><sourceProduct refid="Read"/></sources>
    <parameters>
      <sourceBands>B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12</sourceBands>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>

  <node id="Resample_Reflectance">
    <operator>Resample</operator>
    <sources><sourceProduct refid="Subset_Reflectance"/></sources>
    <parameters>
      <targetResolution>${targetRes}</targetResolution>
      <upsampling>Bicubic</upsampling>
      <downsampling>Median</downsampling>
    </parameters>
  </node>

  <node id="Write_Ref">
    <operator>Write</operator>
    <sources><sourceProduct refid="Resample_Reflectance"/></sources>
    <parameters><file>${outRef}</file></parameters>
  </node>

  <!-- Branch C: SCL (quality_scene_classification) @ targetRes for masking -->
  <node id="Subset_SCL">
    <operator>Subset</operator>
    <sources><sourceProduct refid="Read"/></sources>
    <parameters>
      <sourceBands>quality_scene_classification</sourceBands>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>

  <node id="Resample_SCL">
    <operator>Resample</operator>
    <sources><sourceProduct refid="Subset_SCL"/></sources>
    <parameters>
      <targetResolution>${targetRes}</targetResolution>
      <upsampling>Nearest</upsampling>
      <downsampling>Median</downsampling>
    </parameters>
  </node>

  <node id="Write_SCL">
    <operator>Write</operator>
    <sources><sourceProduct refid="Resample_SCL"/></sources>
    <parameters><file>${outScl}</file></parameters>
  </node>
</graph>
"""

# --- HELPERS ---
RE_DATE = re.compile(r"(\d{8})T\d{6}")
RE_TILE = re.compile(r"(T\d{2}[A-Z]{3})")

def want_tile(name: str, tile_code: str) -> bool:
    mt = RE_TILE.search(name)
    return (mt and mt.group(1) == tile_code)

def fmt_tag(region: str, year: str, dataset: str, tile: str) -> str:
    return f"{region}_{year}0000_{dataset}_{tile}"

def acq_date_from_path(p: Path) -> str:
    """Extract acquisition date from SAFE/SAFE.zip or MTD file."""
    name_for_date = p.parent.name if p.name == "MTD_MSIL2A.xml" else p.name
    m = RE_DATE.search(name_for_date)
    return m.group(1) if m else "unknown"

def safe_stem_from_path(p: Path) -> str:
    """Return clean stem for naming outputs (no .SAFE or .SAFE.zip)."""
    if p.name == "MTD_MSIL2A.xml":
        return p.parent.name.replace(".SAFE", "")
    name = p.name
    if name.endswith(".SAFE.zip"):
        return name.replace(".SAFE.zip", "")
    if name.endswith(".SAFE"):
        return name.replace(".SAFE", "")
    return name

def sensor_from_stem(stem: str) -> str:
    s = stem.upper()
    if s.startswith("S2A_"): return "S2A"
    if s.startswith("S2B_"): return "S2B"
    return "S2A"  # safe default

def orbit_from_name(name: str) -> str:
    """
    Extract orbit code (e.g. R037, R050, R094) from SAFE folder or zip name.
    Returns None if not found.
    """
    m = re.search(r"(R\d{3})", name)
    return m.group(1) if m else None

def ensure_graph_xml(work_dir: Path, xml_text: str) -> Path:
    """
    Write the SNAP graph XML to a deterministic location under WORK.
    Returns the path to the xml file.
    """
    graphs_dir = work_dir / "_graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    xml_path = graphs_dir / "S2_SplitRef_Bio_SCL.xml"
    xml_path.write_text(xml_text, encoding="utf-8")
    return xml_path

def list_inputs_for_plot_tile():
    candidates = []

    # 1) extracted SAFE folders (preferred)
    if EXTRACTED_ROOT.exists():
        for safe_dir in sorted(EXTRACTED_ROOT.glob("S2*_MSIL2A_*.SAFE")):
            if not want_tile(safe_dir.name, TILE):
                continue
            mtd = safe_dir / "MTD_MSIL2A.xml"
            if mtd.exists():
                candidates.append(mtd)

    # 2) zipped SAFE files (fallback, recursive under months)
    if ZIPPED_ROOT.exists():
        for z in sorted(ZIPPED_ROOT.rglob("S2*_MSIL2A_*.SAFE.zip")):
            if not want_tile(z.name, TILE):
                continue
            candidates.append(z)

    if not candidates:
        print(f"⚠️ No inputs found for {PLOT}/{TILE}")
        return []

    # Debug: show raw candidates + orbits
    print(f"   raw candidates for {PLOT}/{TILE}: {len(candidates)}")
    for p in candidates:
        safe_name = p.name if p.suffix == ".zip" else p.parent.name
        print("    ", safe_name, "orbit=", orbit_from_name(safe_name))

    # --- orbit filter ---
    filtered = []
    for p in candidates:
        safe_name = p.name if p.suffix == ".zip" else p.parent.name
        orb = orbit_from_name(safe_name)
        if ORBIT and orb != ORBIT:
            continue
        filtered.append(p)

    if not filtered:
        print(f"⚠️ No inputs left for {PLOT}/{TILE} after orbit filter (ORBIT={ORBIT})")
        return []

    # --- de-duplicate by acquisition date (prefer extracted) ---
    by_date = {}
    for p in filtered:
        d = acq_date_from_path(p)

        # ⛔️ skip known bad dates
        if d in BAD_DATES:
            print(f"   • skipping bad acquisition {d} for {PLOT}/{TILE}")
            continue

        score = 0 if p.name == "MTD_MSIL2A.xml" else 1  # prefer extracted
        if d not in by_date or score < by_date[d][1]:
            by_date[d] = (p, score)

    inputs = [by_date[d][0] for d in sorted(by_date)]
    if not inputs:
        print(f"⚠️ Nothing left after filters (ORBIT={ORBIT}, BAD_DATES={sorted(BAD_DATES)})")
    return inputs

def run_one(input_path: Path):
    stem   = safe_stem_from_path(input_path)
    acq    = acq_date_from_path(input_path)
    sensor = sensor_from_stem(stem)
    graph_xml_path = ensure_graph_xml(WORK, GRAPH_XML_TEXT)


    out_ref = REF_DIR / f"{stem}_{acq}_{TILE}_REF{TARGET_RES}.dim"
    out_bio = BIO_DIR / f"{stem}_{acq}_{TILE}_BIO{TARGET_RES}.dim"
    out_scl = SCL_DIR / f"{stem}_{acq}_{TILE}_SCL{TARGET_RES}.dim"

    # skip if already exists
    if out_ref.exists() and out_bio.exists() and out_scl.exists():
        print(f"   • skip (already have REF/BIO/SCL) for {stem} {acq}")
        return out_ref, out_bio, out_scl

    cmd = [
        GPT_EXE, str(graph_xml_path),
        f"-PinFile={input_path}",
        f"-PoutRef={out_ref}",
        f"-PoutBio={out_bio}",
        f"-PoutScl={out_scl}",
        f"-PtargetRes={TARGET_RES}",
        f"-Psensor={sensor}",
        "-c", f"{MEM_GB}G"
    ]
    print(">>", " ".join(shlex.quote(str(x)) for x in cmd))
    t0 = time.time()
    proc = subprocess.run(cmd, text=True, capture_output=True)
    if proc.stdout: print(proc.stdout)
    if proc.returncode != 0:
        print("---- SNAP ERROR ----")
        if proc.stderr: print(proc.stderr)
        raise subprocess.CalledProcessError(proc.returncode, cmd, output=proc.stdout, stderr=proc.stderr)
    dt = time.time() - t0
    print(f"   ✓ wrote\n     REF: {out_ref}\n     BIO: {out_bio}\n     SCL: {out_scl}  ({dt/60:.1f} min)")
    return out_ref, out_bio, out_scl


# --- RUN ALL ---
inputs = list_inputs_for_plot_tile()
print(f"Found {len(inputs)} S2 L2A inputs for {PLOT}/{TILE}")
for p in inputs:
    print(f"  - {p}   (date={acq_date_from_path(Path(p))})")

ok, fail = 0, 0
for inp in inputs:
    try:
        run_one(Path(inp))
        ok += 1
    except Exception as e:
        fail += 1
        print(f"   ✗ failed on {inp}\n   {e}")

print(f"\nDone. Success: {ok}  Failures: {fail}")
print(f"REF25: {len(list(REF_DIR.glob('*.dim')))}  BIO25: {len(list(BIO_DIR.glob('*.dim')))}  SCL25: {len(list(SCL_DIR.glob('*.dim')))}")


   raw candidates for Teruel/T30TXK: 6
     S2A_MSIL2A_20220828T104631_N0510_R051_T30TXK_20240708T151011.SAFE.zip orbit= R051
     S2B_MSIL2A_20220724T104629_N0510_R051_T30TXK_20240710T190140.SAFE.zip orbit= R051
     S2A_MSIL2A_20220619T104631_N0510_R051_T30TXK_20240628T024845.SAFE.zip orbit= R051
     S2B_MSIL2A_20220515T104619_N0510_R051_T30TXK_20240620T040133.SAFE.zip orbit= R051
     S2B_MSIL2A_20221002T104759_N0510_R051_T30TXK_20240724T091405.SAFE.zip orbit= R051
     S2A_MSIL2A_20220927T104821_N0510_R051_T30TXK_20240726T195157.SAFE.zip orbit= R051
Found 6 S2 L2A inputs for Teruel/T30TXK
  - M:\Project BLS\S2\Data\Aragon\Teruel\2022\May\S2B_MSIL2A_20220515T104619_N0510_R051_T30TXK_20240620T040133.SAFE.zip   (date=20220515)
  - M:\Project BLS\S2\Data\Aragon\Teruel\2022\June\S2A_MSIL2A_20220619T104631_N0510_R051_T30TXK_20240628T024845.SAFE.zip   (date=20220619)
  - M:\Project BLS\S2\Data\Aragon\Teruel\2022\July\S2B_MSIL2A_20220724T104629_N0510_R051_T30TXK_20240710T190140.SAFE.zip  

In [13]:
# --- BIO temporal composite via SNAP GPT ---

BIO25 = BIO_DIR                    # inputs: per-scene BIO .dim
BIO_TC_DIR = WORK / "BIO_TC"        # intermediate composite .dim
BIO_TC_DIR.mkdir(parents=True, exist_ok=True)

# Deliverables: final BigTIFF only
DELIV_BIO = DELIV / "BIO"
DELIV_BIO.mkdir(parents=True, exist_ok=True)

OUT_DIM = BIO_TC_DIR / f"{PLOT}_{TILE}_S2BIOTC.dim"                          # intermediate
OUT_TIF = DELIV_BIO / f"{fmt_tag(REGION_TAG, YEAR, 'S2BIOTC', TILE)}.tif"    # deliverable

WRITE_TIF = True
SKIP_IF_EXISTS = True

bio_dims = sorted(BIO25.glob("*.dim"))
assert bio_dims, f"No BIO .dim files found in {BIO25}"
if not Path(GPT_EXE).exists():
    raise FileNotFoundError(f"GPT not found at: {GPT_EXE}")

def list_bands_from_dim(dim_path: Path):
    txt = dim_path.read_text(encoding="utf-8", errors="ignore")
    root = ET.fromstring(txt); names = []
    for elem in root.iter():
        tagu = elem.tag.upper()
        if tagu.endswith("BAND_NAME") and elem.text:
            names.append(elem.text.strip())
    if not names:
        for elem in root.iter():
            if elem.tag.upper().endswith("BAND"):
                nm = elem.attrib.get("name")
                if nm: names.append(nm.strip())
    return names

def detect_bio_bandnames(dim_path: Path):
    names = [b.strip() for b in list_bands_from_dim(dim_path)]
    lower = {n.lower(): n for n in names}
    def pick(cands):
        for c in cands:
            if c.lower() in lower: return lower[c.lower()]
        return None
    mapping = {
        "LAI":    pick(["lai","LAI"]),
        "FAPAR":  pick(["fapar","FAPAR"]),
        "FCOVER": pick(["fcover","Fcover","FCOVER","fvc","FVC"]),
        "Cab":    pick(["lai_cab","cab","Cab","CAB"]),
        "Cw":     pick(["lai_cw","cw","Cw","CW","cwc","CWC"]),
    }
    missing = [k for k,v in mapping.items() if v is None]
    if missing:
        short = names[:12] + (["..."] if len(names) > 12 else [])
        raise RuntimeError(f"{dim_path.name}: missing bands {missing}. Found: {short}")
    return mapping

def cdata(s: str) -> str: return f"<![CDATA[{s}]]>"

def expr_avg(prefix: str, n_inputs: int) -> str:
    names = [f"{prefix}_{k}" for k in range(1, n_inputs + 1)]
    val   = [f"(({nm}=={nm}) && ({nm}!={NODATA}))" for nm in names]
    num   = " + ".join([f"({v}?{nm}:0)" for v,nm in zip(val,names)])
    den   = " + ".join([f"({v}?1:0)"     for v in val])
    return f"(({den})>0) ? (({num})/({den})) : NaN"

# ----- build XML -----
read_nodes_xml, std_nodes_xml, merge_sources = [], [], []
for i, p in enumerate(bio_dims, start=1):
    rid, std = f"Read{i}", f"Std{i}"
    m = detect_bio_bandnames(p)
    read_nodes_xml.append(f"""
  <node id="{rid}">
    <operator>Read</operator>
    <parameters><file>{p}</file></parameters>
  </node>""")
    std_bands = "\n".join([
f"""      <targetBand><name>LAI_{i}</name><type>float32</type><expression>{cdata(m['LAI'])}</expression><noDataValue>NaN</noDataValue></targetBand>""",
f"""      <targetBand><name>FAPAR_{i}</name><type>float32</type><expression>{cdata(m['FAPAR'])}</expression><noDataValue>NaN</noDataValue></targetBand>""",
f"""      <targetBand><name>FCOVER_{i}</name><type>float32</type><expression>{cdata(m['FCOVER'])}</expression><noDataValue>NaN</noDataValue></targetBand>""",
f"""      <targetBand><name>Cab_{i}</name><type>float32</type><expression>{cdata(m['Cab'])}</expression><noDataValue>NaN</noDataValue></targetBand>""",
f"""      <targetBand><name>Cw_{i}</name><type>float32</type><expression>{cdata(m['Cw'])}</expression><noDataValue>NaN</noDataValue></targetBand>"""
    ])
    std_nodes_xml.append(f"""
  <node id="{std}">
    <operator>BandMaths</operator>
    <sources><sourceProduct refid="{rid}"/></sources>
    <parameters><targetBands>
{std_bands}
    </targetBands><variables/></parameters>
  </node>""")
    merge_sources.append(f'      <sourceProduct{"" if i==1 else f".{i-1}"} refid="{std}"/>')

bandmerge_xml = f"""
  <node id="BandMerge">
    <operator>BandMerge</operator>
    <sources>
{chr(10).join(merge_sources)}
    </sources>
    <parameters><geographicError>1.0E-5</geographicError></parameters>
  </node>"""

tc_targets = []
for prefix, outname in [("LAI","LAI_TC"), ("FAPAR","FAPAR_TC"), ("FCOVER","FCOVER_TC"), ("Cab","Cab_TC"), ("Cw","Cw_TC")]:
    tc_targets.append(f"""
      <targetBand>
        <name>{outname}</name><type>float32</type>
        <expression>{cdata(expr_avg(prefix, len(bio_dims)))}</expression>
        <noDataValue>NaN</noDataValue>
      </targetBand>""")

bandmaths_tc_xml = f"""
  <node id="BandMaths_TC">
    <operator>BandMaths</operator>
    <sources><sourceProduct refid="BandMerge"/></sources>
    <parameters><targetBands>
        {''.join(tc_targets)}
    </targetBands><variables/></parameters>
  </node>"""

write_dim_xml = f"""
  <node id="Write_DIM">
    <operator>Write</operator>
    <sources><sourceProduct refid="BandMaths_TC"/></sources>
    <parameters>
      <file>{OUT_DIM}</file>
      <formatName>BEAM-DIMAP</formatName>
    </parameters>
  </node>"""

write_tif_xml = f"""
  <node id="Write_TIF">
    <operator>Write</operator>
    <sources><sourceProduct refid="BandMaths_TC"/></sources>
    <parameters>
      <file>{OUT_TIF}</file>
      <formatName>GeoTIFF-BigTIFF</formatName>
    </parameters>
  </node>""" if WRITE_TIF else ""

graph_xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<graph id="BIO_TemporalComposite_{TILE}">
  <version>1.0</version>
  {''.join(read_nodes_xml)}
  {''.join(std_nodes_xml)}
  {bandmerge_xml}
  {bandmaths_tc_xml}
  {write_dim_xml}
  {write_tif_xml}
</graph>
"""

# Save graph to global Graphs dir
graphs_dir = WORK / "_graphs"
graphs_dir.mkdir(parents=True, exist_ok=True)
gpath = graphs_dir / f"S2_BIO_TC_{PLOT}_{TILE}.xml"
gpath.write_text(graph_xml, encoding="utf-8")
print("Graph saved to:", gpath)
print("Inputs:", len(bio_dims), "BIO products")
print("Intermediate DIM:", OUT_DIM)
if WRITE_TIF: print("Deliverable BigTIFF:", OUT_TIF)

# Skip guard (don’t re-run if both outputs already exist)
if SKIP_IF_EXISTS and OUT_DIM.exists() and (not WRITE_TIF or OUT_TIF.exists()):
    print("• Skip BIO TC (exists)")
else:
    cmd = [GPT_EXE, str(gpath), "-c", f"{MEM_GB}G"]
    print("Running:", " ".join(cmd))
    t0 = time.time()
    p = subprocess.run(cmd, capture_output=True, text=True)
    if p.stdout.strip(): print("\n[GPT stdout]\n", p.stdout)
    if p.returncode != 0:
        print("\n[GPT stderr]\n", p.stderr)
        raise RuntimeError("BIO temporal composite failed")
    print(f"✓ BIO temporal composite complete in {(time.time()-t0)/60:.1f} min")


Graph saved to: A:\S2\work\Teruel\T30TXK\_graphs\S2_BIO_TC_Teruel_T30TXK.xml
Inputs: 6 BIO products
Intermediate DIM: A:\S2\work\Teruel\T30TXK\BIO_TC\Teruel_T30TXK_S2BIOTC.dim
Deliverable BigTIFF: A:\S2\Deliverables\Teruel\T30TXK\BIO\Teruel_20220000_S2BIOTC_T30TXK.tif
Running: C:\Program Files\esa-snap\bin\gpt.exe A:\S2\work\Teruel\T30TXK\_graphs\S2_BIO_TC_Teruel_T30TXK.xml -c 8G

[GPT stdout]
 Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.

✓ BIO temporal composite complete in 7.6 min


In [15]:
#collocation

RE_ACQ = re.compile(r"(\d{8})T\d{6}")

def _acq(p: Path) -> str:
    m = RE_ACQ.search(p.name)
    return m.group(1) if m else "unknown"

def collect_tile_products(plot: str, tile: str):
    ref_dir = WORK / "REF25"
    scl_dir = WORK / "SCL25"
    refs = list(ref_dir.glob("*.dim"))
    scls = list(scl_dir.glob("*.dim"))
    if not refs or not scls:
        raise RuntimeError(f"No REF/SCL found under {ref_dir} / {scl_dir}")
    ref_by_date = { _acq(p): p for p in refs }
    scl_by_date = { _acq(p): p for p in scls }
    dates = sorted(set(ref_by_date) & set(scl_by_date))
    return [(d, ref_by_date[d], scl_by_date[d]) for d in dates]

def write_collocate_stack_graph(plot: str, tile: str, pairs: list, out_dim: Path) -> Path:
    """
    pairs = [(date, ref_path, scl_path), ...] in chronological order.
    For each date: Merge(REF, SCL) so SCL travels with reflectance.
    Then Collocate: master = Merge0, slaves = Merge1..MergeN-1.
    """
    assert len(pairs) >= 2, "Need at least 2 dates to collocate"
    if len(pairs) < 2:
        raise RuntimeError(f"Need >=2 dates to collocate, found {len(pairs)} for {PLOT}/{TILE}")

    reads_xml = []
    merges_xml = []

    # Build Read + Merge per date
    for i, (d, refp, sclp) in enumerate(pairs):
        reads_xml.append(f"""
  <node id="Read_REF{i}">
    <operator>Read</operator>
    <parameters>
      <file>{refp}</file>
    </parameters>
  </node>
  <node id="Read_SCL{i}">
    <operator>Read</operator>
    <parameters>
      <file>{sclp}</file>
    </parameters>
  </node>""")

        # ✅ Merge has NO <parameters> block — just masterProduct + sourceProduct.n
        merges_xml.append(f"""
  <node id="Merge{i}">
    <operator>Merge</operator>
    <sources>
      <masterProduct>Read_REF{i}</masterProduct>
      <sourceProduct.1>Read_SCL{i}</sourceProduct.1>
    </sources>
  </node>""")

    # Collocate sources: master=Merge0, slaves=Merge1..MergeN-1
    coll_src = ["<master>Merge0</master>"]
    for i in range(1, len(pairs)):
        coll_src.append(f"<slave{i}>Merge{i}</slave{i}>")
    coll_src_xml = "\n      ".join(coll_src)

    graph = f"""<graph id="S2_CollocateOnly_{plot}_{tile}">
  <version>1.0</version>
  {''.join(reads_xml)}
  {''.join(merges_xml)}
  <node id="Collocate">
    <operator>Collocate</operator>
    <sources>
      {coll_src_xml}
    </sources>
    <parameters>
      <copySecondaryMetadata>false</copySecondaryMetadata>
      <renameReferenceComponents>true</renameReferenceComponents>
      <renameSecondaryComponents>true</renameSecondaryComponents>
      <referenceComponentPattern>${{ORIGINAL_NAME}}_M</referenceComponentPattern>
      <secondaryComponentPattern>${{ORIGINAL_NAME}}_S${{SLAVE_NUMBER_ID}}</secondaryComponentPattern>
      <resamplingType>BICUBIC_CONVOLUTION</resamplingType>
    </parameters>
  </node>
  <node id="Write">
    <operator>Write</operator>
    <sources>
      <source>Collocate</source>
    </sources>
    <parameters>
      <file>{out_dim}</file>
    </parameters>
  </node>
</graph>"""

    graphs_dir = WORK / "_graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    gp = graphs_dir / f"S2_CollocateOnly_{plot}_{tile}.xml"
    gp.write_text(graph, encoding="utf-8")
    return gp

def run_collocate_stack(plot: str, tile: str, MEM_GB):
    pairs = collect_tile_products(plot, tile)
    colloc_dir = WORK / "COLLOC"
    colloc_dir.mkdir(parents=True, exist_ok=True)
    out_dim = colloc_dir / f"{plot}_{tile}_collocate_only.dim"
    gp = write_collocate_stack_graph(plot, tile, pairs, out_dim)
    print("Graph:", gp)
    print(">>", GPT_EXE, str(gp))
    t0 = time.time()
    p = subprocess.run([GPT_EXE, str(gp), "-c", f"{MEM_GB}G"], text=True, capture_output=True)
    if p.stdout:
        print(p.stdout)
    if p.returncode != 0:
        print(p.stderr)
        raise RuntimeError("Collocate-only graph failed")
    print(f"✓ Collocate-only → {out_dim}  ({(time.time()-t0)/60:.1f} min)")
    return str(out_dim)

# Example run:
colloc_dim = run_collocate_stack(PLOT, TILE, MEM_GB)
colloc_dim


Graph: A:\S2\work\Teruel\T30TXK\_graphs\S2_CollocateOnly_Teruel_T30TXK.xml
>> C:\Program Files\esa-snap\bin\gpt.exe A:\S2\work\Teruel\T30TXK\_graphs\S2_CollocateOnly_Teruel_T30TXK.xml
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.

✓ Collocate-only → A:\S2\work\Teruel\T30TXK\COLLOC\Teruel_T30TXK_collocate_only.dim  (11.7 min)


'A:\\S2\\work\\Teruel\\T30TXK\\COLLOC\\Teruel_T30TXK_collocate_only.dim'

In [17]:
#Temporal composite
# --- CONFIG ---

COLLOC_DIM = WORK / "COLLOC" / f"{PLOT}_{TILE}_collocate_only.dim"   # your collocate-only output
OUT_DIR    = WORK / "COMPOSITE25"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIM    = OUT_DIR / f"{PLOT}_{TILE}_REF25_TC.dim"

SUFFIX_RE = re.compile(r"_(M|S\d+)$")

def list_bandnames_from_dim(dim_path: Path):
    txt = dim_path.read_text(encoding="utf-8", errors="ignore")
    root = ET.fromstring(txt)
    names = []
    for elem in root.iter():
        tagu = elem.tag.upper()
        if tagu.endswith("BAND_NAME") and elem.text:
            names.append(elem.text.strip())
    return names

def detect_suffixes_and_scl_base(dim_path: Path):
    """
    Returns (suffixes_list, scl_base).
    - suffixes_list like ["M","S0","S1"] from what's actually present
    - scl_base like "quality_scene_classification" detected from band names
    """
    names = list_bandnames_from_dim(dim_path)
    # 1) suffixes present
    suffixes = sorted({SUFFIX_RE.search(n).group(1) for n in names if SUFFIX_RE.search(n)},
                      key=lambda s: (s!="M", int(s[1:]) if s.startswith("S") else -1))
    if not suffixes:
        raise RuntimeError("Could not detect any collocate suffixes (_M, _S0, ...).")

    # 2) detect SCL base by looking for known prefixes + detected suffixes
    scl_candidates = set()
    for n in names:
        m = SUFFIX_RE.search(n)
        if not m:
            continue
        base = n[:m.start()]
        low = base.lower()
        if any(k in low for k in ["scene_class", "quality", "scl"]):
            scl_candidates.add(base)

    if not scl_candidates:
        raise RuntimeError("Could not find an SCL band base name in the collocate product.")

    # Prefer the longest (most explicit) base
    scl_base = sorted(scl_candidates, key=len, reverse=True)[0]
    return suffixes, scl_base


# Bands and masks
SCL_BAD_VALUES = [0,1,3,7,8,9,10,11]
SUFFIXES, SCL_BASE = detect_suffixes_and_scl_base(COLLOC_DIM)
print("Detected suffixes:", SUFFIXES)
print("Detected SCL base:", SCL_BASE)

# SCL classes to EXCLUDE: 0=No data, 1=Saturated/defective, 3=Cloud shadows,
# 7=Unclassified, 8=Cloud medium prob, 9=Cloud high prob, 10=Thin cirrus, 11=Snow/ice

def _mask_expr(scl_name: str) -> str:
    terms = [f"({scl_name}!={v})" for v in SCL_BAD_VALUES]
    return " && ".join(terms)

def _band_mean_expr(band: str, suffixes: list[str]) -> str:
    num_terms, den_terms = [], []
    for suf in suffixes:
        bname = f"{band}_{suf}"
        scl   = f"{SCL_BASE}_{suf}"
        valid = f"({_mask_expr(scl)})"
        num_terms.append(f"({bname} * ({valid} ? 1 : 0))")
        den_terms.append(f"(({valid}) ? 1 : 0)")
    num = " + ".join(num_terms)
    den = " + ".join(den_terms)
    return f"(({den}) > 0) ? ({num})/({den}) : NaN"

def write_tc_from_collocated(colloc_dim: Path, out_dim: Path) -> Path:
    tgt_bands = []
    for b in REF_BANDS:
        expr = _band_mean_expr(b, SUFFIXES)
        tgt_bands.append(
            f"""
            <targetBand>
              <name>{b}_TC</name>
              <type>float32</type>
              <expression><![CDATA[{expr}]]></expression>
              <noDataValue>NaN</noDataValue>
            </targetBand>"""
        )
    graph = f"""<graph id="S2_TC_from_collocated_{PLOT}_{TILE}">
  <version>1.0</version>

  <node id="Read">
    <operator>Read</operator>
    <parameters>
      <file>{colloc_dim}</file>
    </parameters>
  </node>

  <node id="BandMaths_TC">
    <operator>BandMaths</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters>
      <targetBands>
        {''.join(tgt_bands)}
      </targetBands>
    </parameters>
  </node>

  <node id="Write">
    <operator>Write</operator>
    <sources>
      <sourceProduct refid="BandMaths_TC"/>
    </sources>
    <parameters>
      <file>{out_dim}</file>
    </parameters>
  </node>
</graph>"""
    
    graphs_dir = WORK / "_graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    gp = graphs_dir / f"S2_TC_from_collocated_{PLOT}_{TILE}.xml"
    gp.write_text(graph, encoding="utf-8")
    return gp

def run_tc_from_collocated():
    assert COLLOC_DIM.exists(), f"Collocate-only product not found: {COLLOC_DIM}"
    gp = write_tc_from_collocated(COLLOC_DIM, OUT_DIM)
    print("Graph:", gp)
    cmd = [GPT_EXE, str(gp), "-c", "8G"]
    print(">>", " ".join(map(str, cmd)))
    t0 = time.time()
    p = subprocess.run(cmd, text=True, capture_output=True)
    if p.stdout: print(p.stdout)
    if p.returncode != 0:
        print(p.stderr)
        raise RuntimeError("Temporal composite (BandMaths-only) failed")
    print(f"✓ Temporal composite → {OUT_DIM}  ({(time.time()-t0)/60:.1f} min)")
    return str(OUT_DIM)

run_tc_from_collocated()


Detected suffixes: ['M', 'S0', 'S1', 'S2', 'S3', 'S4']
Detected SCL base: quality_scene_classification
Graph: A:\S2\work\Teruel\T30TXK\_graphs\S2_TC_from_collocated_Teruel_T30TXK.xml
>> C:\Program Files\esa-snap\bin\gpt.exe A:\S2\work\Teruel\T30TXK\_graphs\S2_TC_from_collocated_Teruel_T30TXK.xml -c 8G
Executing processing graph
....10%....21%....32%....42%....53%....63%....73%....84%... done.

✓ Temporal composite → A:\S2\work\Teruel\T30TXK\COMPOSITE25\Teruel_T30TXK_REF25_TC.dim  (3.3 min)


'A:\\S2\\work\\Teruel\\T30TXK\\COMPOSITE25\\Teruel_T30TXK_REF25_TC.dim'

In [21]:
#GLCM

TC_DIM  = COMPOSITE_DIR / f"{PLOT}_{TILE}_REF25_TC.dim"

OUT_DIR = GLCM_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIM = OUT_DIR / f"{PLOT}_{TILE}_GLCM_5x5_64gl_ALL_ALLBANDS.dim"

WINDOW     = "5x5"
ANGLES     = "ALL"
QUANTIZER  = "Probabilistic Quantizer"
GREYLEVELS = "64"
DISP       = "1"
NODATA     = "-9999.0"

def write_glcm_allbands_graph(tc_dim: Path, out_dim: Path) -> Path:
    # Read node
    read_xml = f"""
  <node id="ReadTC">
    <operator>Read</operator>
    <parameters>
      <file>{tc_dim}</file>
    </parameters>
  </node>"""

    # One GLCM node per band
    glcm_nodes = []
    merge_sources = []
    for i, band in enumerate(TC_BANDS, start=1):
        node_id = f"GLCM_{band}"
        glcm_nodes.append(f"""
  <node id="{node_id}">
    <operator>GLCM</operator>
    <sources>
      <sourceProduct refid="ReadTC"/>
    </sources>
    <parameters>
      <sourceBands>{band}</sourceBands>
      <windowSizeStr>{WINDOW}</windowSizeStr>
      <angleStr>{ANGLES}</angleStr>
      <quantizerStr>{QUANTIZER}</quantizerStr>
      <quantizationLevelsStr>{GREYLEVELS}</quantizationLevelsStr>
      <displacement>{DISP}</displacement>
      <noDataValue>{NODATA}</noDataValue>

      <outputContrast>true</outputContrast>
      <outputDissimilarity>true</outputDissimilarity>
      <outputHomogeneity>true</outputHomogeneity>
      <outputASM>true</outputASM>
      <outputEnergy>true</outputEnergy>
      <outputMAX>true</outputMAX>
      <outputEntropy>true</outputEntropy>
      <outputMean>true</outputMean>
      <outputVariance>true</outputVariance>
      <outputCorrelation>true</outputCorrelation>
    </parameters>
  </node>""")
        merge_sources.append(f'      <sourceProduct.{i} refid="{node_id}"/>')

    # Merge all GLCM outputs
    merge_xml = f"""
  <node id="BandMerge">
    <operator>BandMerge</operator>
    <sources>
{chr(10).join(merge_sources)}
    </sources>
    <parameters>
      <geographicError>1.0E-5</geographicError>
    </parameters>
  </node>"""

    # Write
    write_xml = f"""
  <node id="Write">
    <operator>Write</operator>
    <sources>
      <sourceProduct refid="BandMerge"/>
    </sources>
    <parameters>
      <file>{out_dim}</file>
    </parameters>
  </node>"""

    graph = f"""<?xml version="1.0" encoding="UTF-8"?>
<graph id="S2_GLCM_AllBands_{PLOT}_{TILE}">
  <version>1.0</version>
  {read_xml}
  {''.join(glcm_nodes)}
  {merge_xml}
  {write_xml}
</graph>"""
    
    graphs_dir = WORK / "_graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    gp = graphs_dir / f"S2_GLCM_AllBands_{PLOT}_{TILE}.xml"
    gp.write_text(graph, encoding="utf-8")
    return gp

def run_glcm_allbands():
    assert TC_DIM.exists(), f"Missing TC: {TC_DIM}"
    gp = write_glcm_allbands_graph(TC_DIM, OUT_DIM)
    print("Graph:", gp)
    cmd = [GPT_EXE, str(gp), "-c", "8G"]
    print(">>", " ".join(map(str, cmd)))
    t0 = time.time()
    p = subprocess.run(cmd, text=True, capture_output=True)
    if p.stdout: print(p.stdout)
    if p.returncode != 0:
        print(p.stderr)
        # If SNAP parameter names differ, check:  "gpt -h GLCM"
        raise RuntimeError("GLCM all-bands graph failed")
    print(f"✓ GLCM (all bands) → {OUT_DIM}  ({(time.time()-t0)/60:.1f} min)")
    return str(OUT_DIM)

run_glcm_allbands()


Graph: A:\S2\work\Teruel\T30TXK\_graphs\S2_GLCM_AllBands_Teruel_T30TXK.xml
>> C:\Program Files\esa-snap\bin\gpt.exe A:\S2\work\Teruel\T30TXK\_graphs\S2_GLCM_AllBands_Teruel_T30TXK.xml -c 8G
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.

✓ GLCM (all bands) → A:\S2\work\Teruel\T30TXK\GLCM25\Teruel_T30TXK_GLCM_5x5_64gl_ALL_ALLBANDS.dim  (16.0 min)


'A:\\S2\\work\\Teruel\\T30TXK\\GLCM25\\Teruel_T30TXK_GLCM_5x5_64gl_ALL_ALLBANDS.dim'

In [23]:
# --- EXPORT STEP ---
# Converts SNAP-native TC product (.dim) into GeoTIFF
# for downstream Python / ML workflows (e.g. PCA, LightGBM).

SRC_DIM   = COMPOSITE_DIR / f"{PLOT}_{TILE}_REF25_TC.dim"

OUT_TIF   = Path(r"A:\S2\Deliverables") / REGION_TAG / TILE / "S2TC" / f"{fmt_tag(REGION_TAG, YEAR, 'S2TC', TILE)}.tif"
OUT_TIF.parent.mkdir(parents=True, exist_ok=True)

# Skip guard
if OUT_TIF.exists():
    print(f"• Skip export (exists): {OUT_TIF}")
else:
    graph = f"""<graph id="Export_TC_TIFF">
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <parameters><file>{SRC_DIM}</file></parameters>
  </node>
  <node id="Subset">
    <operator>Subset</operator>
    <sources><sourceProduct refid="Read"/></sources>
    <parameters>
      <sourceBands>{','.join(TC_BANDS)}</sourceBands>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>
  <node id="Write">
    <operator>Write</operator>
    <sources><sourceProduct refid="Subset"/></sources>
    <parameters>
      <file>{OUT_TIF}</file>
      <formatName>GeoTIFF</formatName>
      <!-- If needed for size: use GeoTIFF-BigTIFF -->
    </parameters>
  </node>
</graph>"""

    graphs_dir = WORK / "_graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    gx = graphs_dir / f"S2_ExportTC_{PLOT}_{TILE}.xml"
    gx.write_text(graph, encoding="utf-8")

    print("Graph:", gx)
    t0 = time.time()
    p = subprocess.run([GPT_EXE, str(gx), "-c", "8G"], text=True, capture_output=True)
    print(p.stdout)
    if p.returncode != 0:
        print(p.stderr)
        raise RuntimeError("Export to GeoTIFF failed")
    print(f"✓ Wrote {OUT_TIF}  in {(time.time()-t0)/60:.1f} min")
    print("Exists?", OUT_TIF.exists())


Graph: A:\S2\work\Teruel\T30TXK\_graphs\S2_ExportTC_Teruel_T30TXK.xml
Executing processing graph
....10%....21%....32%....42%....53%....63%....73%....84%... done.

✓ Wrote A:\S2\Deliverables\Teruel\T30TXK\S2TC\Teruel_20220000_S2TC_T30TXK.tif  in 0.7 min
Exists? True


In [27]:
# === EXPORT & DOCUMENTATION (GeoTIFF + README, dim-first) ===

SKIP_IF_EXISTS = True

# ---------- STATIC BAND MAPS (fallbacks only) ----------
BAND_MAPS = {
    # S2 temporal composite (12 bands)
    "S2TC": [
        "B1 (Aerosol, 443 nm)",
        "B2 (Blue, 490 nm)",
        "B3 (Green, 560 nm)",
        "B4 (Red, 665 nm)",
        "B5 (705 nm)",
        "B6 (740 nm)",
        "B7 (783 nm)",
        "B8 (NIR, 842 nm)",
        "B8A (865 nm)",
        "B9 (940 nm)",
        "B11 (SWIR 1610 nm)",
        "B12 (SWIR 2190 nm)",
    ],
    # BIO temporal composite (5): *_TC bands
    "BIO": ["LAI_TC","FAPAR_TC","FCOVER_TC","Cab_TC","Cw_TC"],
    # BIO per-date (10): value + flags for each variable, order matches your SNAP chain
    "BIO10": [
        "LAI","LAI_flags",
        "Cab","Cab_flags",
        "Cw","Cw_flags",
        "FAPAR","FAPAR_flags",
        "FCOVER","FCOVER_flags",
    ],
    # SCL (kept here if you ever export SCL to GeoTIFF)
    "SCL": [
        "0 = No data","1 = Saturated/defective","2 = Dark area pixels",
        "3 = Cloud shadows","4 = Vegetation","5 = Bare soils","6 = Water",
        "7 = Clouds low probability","8 = Clouds medium probability",
        "9 = Clouds high probability","10 = Cirrus","11 = Snow / ice",
    ],
    # GLCM name synthesis (stat list)
    "GLCM_STATS": [
        "Contrast","Dissimilarity","Homogeneity","ASM","Energy",
        "MAX","Entropy","GLCMMean","GLCMVariance","GLCMCorrelation"
    ],
}

# ---------- SNAP helpers ----------
def write_graph_read_to_geotiff(src_dim: Path, dst_tif: Path, bigtiff=False) -> Path:
    """Tiny SNAP graph: Read -> Write(GeoTIFF/BigTIFF). Returns .xml path."""
    fmt = "GeoTIFF-BigTIFF" if bigtiff else "GeoTIFF"
    graph = f"""<graph id="ReadToGTiff">
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <parameters><file>{src_dim}</file></parameters>
  </node>
  <node id="Write">
    <operator>Write</operator>
    <sources><sourceProduct refid="Read"/></sources>
    <parameters>
      <file>{dst_tif}</file>
      <formatName>{fmt}</formatName>
    </parameters>
  </node>
</graph>"""
    gp = src_dim.with_suffix(".read2tif.xml")
    gp.write_text(graph, encoding="utf-8")
    return gp

def run_gpt(graph_path: Path, MEM_GB):
    cmd = [GPT_EXE, str(graph_path), "-c", f"{MEM_GB}G"]
    print(">>", " ".join(shlex.quote(str(x)) for x in cmd))
    t0 = time.time()
    p = subprocess.run(cmd, text=True, capture_output=True)
    if p.stdout: print(p.stdout)
    if p.returncode != 0:
        print(p.stderr)
        raise RuntimeError(f"GPT failed on {graph_path.name}")
    print(f"✓ Wrote via {graph_path.name} in {(time.time()-t0)/60:.1f} min")

# ---------- .dim band-name extractor (robust, simple) ----------
def extract_band_names_from_dim(dim_path: Path) -> list[str]:
    """Return band names in order from SNAP .dim; empty list if not found/parsable."""
    if not dim_path or not dim_path.exists():
        return []
    try:
        root = ET.parse(dim_path).getroot()
    except Exception:
        return []
    # 1) <BandList>/<Band>/<Name>
    names = [b.findtext("Name") for b in root.findall(".//BandList/Band")]
    names = [n for n in names if n]
    if names: return names
    # 2) Fallbacks
    alt = []
    for b in root.findall(".//Band"):
        nm = b.get("name") or b.findtext("Name") or b.findtext("name")
        if nm: alt.append(nm)
    if alt: return alt
    alt = []
    for rdn in root.findall(".//RasterDataNode"):
        nm = rdn.findtext("name")
        if nm: alt.append(nm)
    if alt: return alt
    alt = []
    for sbi in root.findall(".//Spectral_Band_Info"):
        nm = sbi.get("name") or sbi.findtext("name")
        if nm: alt.append(nm)
    return alt

# ---------- Minimal README writer (no TIFF reads) ----------
def generate_readme_simple(
    plot:str, tile:str, year:str,
    work_tile_dir:Path, deliver_tile_dir:Path,
    region_tag:str=None, crs_hint:str=None, pixel_size_m:int=25,
    include_fallback_notes:bool=True
) -> Path:
    """Writes README_bandmap.txt in deliver_tile_dir using .dim if present, otherwise BAND_MAPS fallbacks."""
    region_tag = region_tag or plot
    if crs_hint is None:
        crs_hint = "EPSG:3763 (Portugal)" if "Serra" in region_tag else "EPSG:25830 (Spain)"

    # Work (for .dim discovery)
    COMPOSITE_DIR = work_tile_dir / "COMPOSITE25"
    BIO_DIR       = work_tile_dir / "BIO25"
    GLCM_DIR      = work_tile_dir / "GLCM25"

    tc_dim   = next(COMPOSITE_DIR.glob("*_REF25_TC.dim"), None) if COMPOSITE_DIR.exists() else None
    bio_dims = sorted(BIO_DIR.glob("*.dim")) if BIO_DIR.exists() else []
    glcm_dim = next(GLCM_DIR.glob("*.dim"), None) if GLCM_DIR.exists() else None

    # Deliverables (decide which sections to document)
    D_S2TC = deliver_tile_dir / "S2TC"
    D_BIO  = deliver_tile_dir / "BIO"
    D_GLCM = deliver_tile_dir / "GLCM"

    tc_tif   = next(D_S2TC.glob(f"{region_tag}_{year}0000_S2TC_{tile}.tif"), None) if D_S2TC.exists() else None
    bio_tifs = sorted(D_BIO.glob("*.tif")) if D_BIO.exists() else []
    glcm_tif = next(D_GLCM.glob(f"{region_tag}_{year}0000_GLCM_{tile}.tif"), None) if D_GLCM.exists() else None

    def write_section(lines, label, names, filelabel, fallback_key=None):
        if include_fallback_notes and fallback_key:
            lines.append(f"(Used fallback band list for {fallback_key})")
        lines.append("-"*64)
        lines.append(label)
        lines.append(f"File: {filelabel}")
        lines.append(f"Bands ({len(names)}):")
        for i, nm in enumerate(names, 1):
            lines.append(f"  band_{i} = {nm}")
        lines.append("")

    dataset_dir = deliver_tile_dir / "DATASET"
    dataset_dir.mkdir(parents=True, exist_ok=True)
    out_txt = dataset_dir / "README_bandmap.txt"
    lines = [
        "README_bandmap.txt","------------------\n",
        f"Region: {region_tag}", f"Tile: {tile}", f"Year: {year}",
        f"Projection: {crs_hint}", f"Resolution: {pixel_size_m} m","",
        "Notes:",
        "- Band names come from .dim when available; otherwise a standard list is used.",
        "- GeoTIFF band order mirrors the source .dim order; TIFF metadata was not read.",
        "- *_CLIP.tif keep the same band order as their originals.",""
    ]

    # S2TC
    if tc_tif:
        names = extract_band_names_from_dim(tc_dim) if tc_dim else []
        write_section(lines, "S2TC (Reflectance temporal composite)",
                      names if names else BAND_MAPS["S2TC"], tc_tif.name,
                      None if names else "S2TC")

    # BIO (split composite vs per-date by filename)
    if bio_tifs:
        bio_tc = [t for t in bio_tifs if re.search(r"S2BIOTC|_BIO25_TC", t.name, re.I)]
        bio_dt = [t for t in bio_tifs if t not in bio_tc]

        for t in bio_tc:
            dim = next((d for d in bio_dims if d.stem == t.stem[:-4]), None)
            names = extract_band_names_from_dim(dim) if dim else []
            write_section(lines, "BIO (Biophysical temporal composite)",
                          names if names else BAND_MAPS["BIO"], t.name,
                          None if names else "BIO")

        for t in bio_dt:
            dim = next((d for d in bio_dims if d.stem == t.stem[:-4]), None)
            names = extract_band_names_from_dim(dim) if dim else []
            write_section(lines, "BIO (Per-date biophysical product)",
                          names if names else BAND_MAPS["BIO10"], t.name,
                          None if names else "per-date BIO")

    # GLCM
    if glcm_tif:
        names = extract_band_names_from_dim(glcm_dim) if glcm_dim else []
        if not names:
            # synthesize 12 S2TC bands × 10 stats = 120
            base = [b.split()[0] for b in BAND_MAPS["S2TC"]]  # "B1 (Aerosol..." -> "B1"
            stats = BAND_MAPS["GLCM_STATS"]
            names = [f"{b}_{s}" for b in base for s in stats]
            write_section(lines, "GLCM (Texture features from REF_TC)", names, glcm_tif.name, "GLCM")
        else:
            write_section(lines, "GLCM (Texture features from REF_TC)", names, glcm_tif.name)

    out_txt.write_text("\n".join(lines), encoding="utf-8")
    print(f"✓ Wrote {out_txt}")
    return out_txt

# ---------- Locate inputs/outputs ----------
ROOT = WORK
COMPOSITE_DIR = ROOT / "COMPOSITE25"
BIO_DIR       = ROOT / "BIO25"
GLCM_DIR      = ROOT / "GLCM25"

# Deliverables
D_TILE   = DELIV 
D_S2TC   = D_TILE / "S2TC"
D_BIO    = D_TILE / "BIO"
D_GLCM   = D_TILE / "GLCM"
for d in (D_S2TC, D_BIO, D_GLCM):
    d.mkdir(parents=True, exist_ok=True)

# Expected products
tc_src = next(COMPOSITE_DIR.glob("*_REF25_TC.dim"), None)
tc_tif = D_S2TC / f"{fmt_tag(REGION_TAG, YEAR, 'S2TC', TILE)}.tif"

bio_dims = sorted(BIO_DIR.glob("*.dim"))
bio_outs = [D_BIO / (p.stem + ".tif") for p in bio_dims]

glcm_src = next(GLCM_DIR.glob("*.dim"), None)
glcm_tif = D_GLCM / f"{fmt_tag(REGION_TAG, YEAR, 'GLCM', TILE)}.tif" if glcm_src else None

print("== Plan ==")
print("TC dim :", tc_src)
print("TC tif :", tc_tif)
print("BIO dims:", len(bio_dims))
print("GLCM dim:", glcm_src)
print("GLCM tif:", glcm_tif)
print()

# ---------- Convert (resilient to missing .dim) ----------
# 1) S2TC
if tc_src is None:
    if tc_tif.exists():
        print(f"• No TC .dim, but TIF exists → skipping export: {tc_tif}")
    else:
        print(f"• WARN: No TC .dim and no TIF → skipping S2TC for {PLOT}/{TILE}")
else:
    if tc_tif.exists() and SKIP_IF_EXISTS:
        print(f"• Skip (exists) {tc_tif}")
    else:
        gp = write_graph_read_to_geotiff(tc_src, tc_tif, bigtiff=False)
        run_gpt(gp, MEM_GB)
        print(f"→ {tc_tif}")

# 2) BIO per-date
if bio_dims:
    for src_dim, dst_tif in zip(bio_dims, bio_outs):
        if dst_tif.exists() and SKIP_IF_EXISTS:
            print(f"• Skip (exists) {dst_tif}")
            continue
        gp = write_graph_read_to_geotiff(src_dim, dst_tif, bigtiff=False)
        run_gpt(gp, MEM_GB)
        print(f"→ {dst_tif}")
else:
    print("• No per-date BIO .dim files found — assuming BIO TIFs already exported or not needed.")

# 3) GLCM
if glcm_src is None:
    if glcm_tif and glcm_tif.exists():
        print(f"• No GLCM .dim, but TIF exists → skipping export: {glcm_tif}")
    else:
        print(f"• WARN: No GLCM .dim and no TIF → skipping GLCM for {PLOT}/{TILE}")
else:
    if glcm_tif.exists() and SKIP_IF_EXISTS:
        print(f"• Skip (exists) {glcm_tif}")
    else:
        gp = write_graph_read_to_geotiff(glcm_src, glcm_tif, bigtiff=True)
        run_gpt(gp, MEM_GB)
        print(f"→ {glcm_tif}")

# ---------- README: always generate next to deliverables ----------
_ = generate_readme_simple(
    plot=PLOT, tile=TILE, year=YEAR,
    work_tile_dir=ROOT,
    deliver_tile_dir=D_TILE,
    region_tag=REGION_TAG
)

print("\nAll done.")


== Plan ==
TC dim : A:\S2\work\Teruel\T30TXK\COMPOSITE25\Teruel_T30TXK_REF25_TC.dim
TC tif : A:\S2\Deliverables\Teruel\T30TXK\S2TC\Teruel_20220000_S2TC_T30TXK.tif
BIO dims: 6
GLCM dim: A:\S2\work\Teruel\T30TXK\GLCM25\Teruel_T30TXK_GLCM_5x5_64gl_ALL_ALLBANDS.dim
GLCM tif: A:\S2\Deliverables\Teruel\T30TXK\GLCM\Teruel_20220000_GLCM_T30TXK.tif

• Skip (exists) A:\S2\Deliverables\Teruel\T30TXK\S2TC\Teruel_20220000_S2TC_T30TXK.tif
>> 'C:\Program Files\esa-snap\bin\gpt.exe' 'A:\S2\work\Teruel\T30TXK\BIO25\S2A_MSIL2A_20220619T104631_N0510_R051_T30TXK_20240628T024845_20220619_T30TXK_BIO25.read2tif.xml' -c 8G
Executing processing graph
....10%....21%....32%....43%....54%....65%....75%....86%.. done.

✓ Wrote via S2A_MSIL2A_20220619T104631_N0510_R051_T30TXK_20240628T024845_20220619_T30TXK_BIO25.read2tif.xml in 0.4 min
→ A:\S2\Deliverables\Teruel\T30TXK\BIO\S2A_MSIL2A_20220619T104631_N0510_R051_T30TXK_20240628T024845_20220619_T30TXK_BIO25.tif
>> 'C:\Program Files\esa-snap\bin\gpt.exe' 'A:\S2\work\

In [29]:

# --- CONFIG: locate SCL and where to write QA ---
WORK_ROOT = Path(r"A:\S2\work")
SCL_DIR   = WORK_ROOT / PLOT / TILE / "SCL25"

QA_DIR = Path(r"A:\S2\Deliverables") / REGION_TAG / TILE / "DATASET" / "QA"
QA_DIR.mkdir(parents=True, exist_ok=True)
OUT_CSV = QA_DIR / f"{fmt_tag(REGION_TAG, YEAR, 'SCL_TALLY', TILE)}.csv"

# Sentinel-2 SCL legend (0..11)
SCL_MAP = {
    0:"No data", 1:"Saturated/defective", 2:"Dark features/shadows", 3:"Cloud shadows",
    4:"Vegetation", 5:"Bare soils", 6:"Water", 7:"Unclassified",
    8:"Cloud medium prob.", 9:"Cloud high prob.", 10:"Thin cirrus", 11:"Snow/ice",
}
BAD_CLOUD = {3,8,9,10,11}        # clouds-only fraction
BAD_ALL   = {0,1,3,7,8,9,10,11}  # your masking set (keeps 2,4,5,6)

def resolve_scl_raster(dim_path: Path) -> Path:
    """Given an SCL .dim, return the path to its .img band."""
    if dim_path.suffix.lower() != ".dim":
        return dim_path  # already a raster (e.g., .tif/.img)
    data_dir = dim_path.with_suffix(".data")
    # Typical SNAP band name for SCL:
    cands = list(data_dir.glob("quality_scene_classification*.img"))
    if not cands:
        cands = list(data_dir.glob("*SCL*.img")) + list(data_dir.glob("*.img"))
    if not cands:
        raise FileNotFoundError(f"No .img band files under {data_dir}")
    return cands[0]

def scl_histogram(raster_path: Path):
    counts = {k: 0 for k in SCL_MAP}
    total = 0
    with rasterio.open(raster_path) as ds:
        for _, window in ds.block_windows(1):
            arr = ds.read(1, window=window, masked=True)
            data = arr.compressed().astype(np.int32)
            total += data.size
            if data.size:
                bc = np.bincount(data, minlength=12)
                for k in range(min(12, len(bc))):
                    counts[k] += int(bc[k])
    return counts, total

def write_folder_tally(scl_dir: Path, out_csv: Path):
    dims = sorted(scl_dir.glob("*.dim"))
    if not dims:
        raise FileNotFoundError(f"No .dim files in {scl_dir}")

    out_csv.parent.mkdir(parents=True, exist_ok=True)

    grand_counts = {k: 0 for k in SCL_MAP}
    grand_total = 0

    with out_csv.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["scene", "class", "label", "count", "fraction", "percent",
                    "cloud_frac", "mask_all_frac", "dark_frac"])

        for dim in dims:
            scene = dim.stem
            img = resolve_scl_raster(dim)
            counts, total = scl_histogram(img)
            cloud = sum(counts[k] for k in BAD_CLOUD)
            mask_all = sum(counts[k] for k in BAD_ALL)
            dark = counts.get(2, 0)

            # per-class rows for this scene
            for k in range(12):
                n = counts.get(k, 0)
                frac = (n/total) if total else 0.0
                w.writerow([scene, k, SCL_MAP[k], n, f"{frac:.6f}", f"{100*frac:.2f}",
                            f"{(cloud/total) if total else 0:.6f}",
                            f"{(mask_all/total) if total else 0:.6f}",
                            f"{(dark/total) if total else 0:.6f}"])

            # accumulate for grand summary
            for k in grand_counts:
                grand_counts[k] += counts.get(k, 0)
            grand_total += total

        # aggregated summary
        w.writerow([])
        w.writerow(["ALL_SCENES", "—", "—", "—", "—", "—", "—", "—", "—"])
        cloud = sum(grand_counts[k] for k in BAD_CLOUD)
        mask_all = sum(grand_counts[k] for k in BAD_ALL)
        dark = grand_counts.get(2, 0)
        for k in range(12):
            n = grand_counts.get(k, 0)
            frac = (n/grand_total) if grand_total else 0.0
            w.writerow(["ALL_SCENES", k, SCL_MAP[k], n, f"{frac:.6f}", f"{100*frac:.2f}",
                        f"{(cloud/grand_total) if grand_total else 0:.6f}",
                        f"{(mask_all/grand_total) if grand_total else 0:.6f}",
                        f"{(dark/grand_total) if grand_total else 0:.6f}"])

    return out_csv

# Run it (optional skip guard)
if OUT_CSV.exists():
    print(f"• Skip QA (exists): {OUT_CSV}")
else:
    print("→", write_folder_tally(SCL_DIR, OUT_CSV))

→ A:\S2\Deliverables\Teruel\T30TXK\DATASET\QA\Teruel_20220000_SCL_TALLY_T30TXK.csv


In [33]:
# === S2 DATA BUILDER: split -> name -> reproject (GDAL CLI), write inventory ===

# ---- Helpers ----
def run_cmd(args: list[str]):
    # Pretty-print for logs
    print(">>", " ".join(args))
    p = subprocess.run(args, text=True, capture_output=True)  # shell=False by default
    if p.stdout.strip():
        print(p.stdout.strip())
    if p.returncode != 0:
        if p.stderr.strip():
            print(p.stderr.strip())
        raise RuntimeError("Command failed")

def gdal_translate_band(src_tif: Path, band_idx: int, tmp_tif: Path, nodata=NODATA):
    args = [
        "gdal_translate",
        "-b", str(band_idx),
        "-a_nodata", str(nodata),
        "-of", "GTiff",
        str(src_tif),
        str(tmp_tif),
    ]
    run_cmd(args)

def gdal_warp_reproject(src_tif: Path, dst_tif: Path, epsg: str, nodata=NODATA, resample="bilinear"):
    args = [
        "gdalwarp",
        "-overwrite",
        "-t_srs", epsg,
        "-srcnodata", str(nodata),
        "-dstnodata", str(nodata),
        "-r", resample,
        "-multi",
        str(src_tif),
        str(dst_tif),
    ]
    run_cmd(args)

def ensure_dir(p: Path):
    p.parent.mkdir(parents=True, exist_ok=True)

# ---- Where to read and write ----
TROOT = DELIV  # already = ...\SerraMafra\T29SMD   (fixes duplication)
OUT_BASE = TROOT / "DATASET"
OUT_BASE.mkdir(parents=True, exist_ok=True)
INV_CSV  = OUT_BASE / f"{fmt_tag(REGION_TAG, YEAR, 'S2_INVENTORY', TILE)}.csv"
ensure_dir(INV_CSV)

# -- Robust discovery (prefer non-CLIP, fall back to CLIPPED / recursive) --
def pick_prefer_unclipped(paths):
    # sort so that unclipped come first; if both exist, we use the first
    return sorted(paths, key=lambda p: ("_CLIP" in p.stem, p.name))

# S2TC
S2TC_DIR = TROOT / "S2TC"
s2tc_list = list(S2TC_DIR.glob("*.tif"))
if not s2tc_list:
    s2tc_list = list(TROOT.rglob("*S2TC*.tif"))
s2tc_list = pick_prefer_unclipped(s2tc_list)
SRC_S2TC = s2tc_list[0] if s2tc_list else None

# BIO (per-date + BIOTC)
BIO_DIR = TROOT / "BIO"
bio_list = list(BIO_DIR.glob("*.tif"))
if not bio_list:
    bio_list = list(TROOT.rglob("*BIO*.tif"))
# split composite vs per-date, keep stable order
bio_tc   = [p for p in bio_list if re.search(r"(S2BIOTC|_BIO25_TC)", p.name, re.I)]
bio_date = [p for p in bio_list if p not in bio_tc]
bio_tc   = pick_prefer_unclipped(bio_tc)
bio_date = pick_prefer_unclipped(bio_date)
SRC_BIO  = bio_tc + bio_date  # process composite first, then dates

# GLCM
GLCM_DIR = TROOT / "GLCM"
glcm_list = list(GLCM_DIR.glob("*.tif"))
if not glcm_list:
    glcm_list = list(TROOT.rglob("*GLCM*.tif"))
glcm_list = pick_prefer_unclipped(glcm_list)
SRC_GLCM = glcm_list[0] if glcm_list else None

print("== S2 Data Builder ==")
print("• S2TC:", [SRC_S2TC.name] if SRC_S2TC else [])
print("• BIO_TC:", [p.name for p in bio_tc])
print("• BIO per-date:", len(bio_date))
print("• GLCM:", [SRC_GLCM.name] if SRC_GLCM else [])

# ---- Build splitting plans ----
def plan_tc(src: Path):
    return [(i, var, OUT_BASE / out_name(REGION_TAG, YEAR, PLATFORM, var, TILE), RESAMPLE_NUMERIC)
            for i, var in enumerate(TC_DATASET_ORDER, start=1)]

def plan_bio(src: Path):
    is_tc = bool(re.search(r"(S2BIOTC|_BIO25_TC)", src.name, re.I))
    order = BIO_TC_ORDER if is_tc else BIO_DT_ORDER
    plan = []
    for i, var in enumerate(order, start=1):
        res = RESAMPLE_FLAGS if var.endswith("FLAGS") else RESAMPLE_NUMERIC
        plan.append((i, var, OUT_BASE / out_name(REGION_TAG, YEAR, PLATFORM, var, TILE), res))
    return plan

def plan_glcm(src: Path):
    plan = []
    band_idx = 1
    for b in TC_DATASET_ORDER:                 # B01..B12
        for stat in GLCM_STATS:        # 10 stats
            var = f"{b}{stat}"         # e.g., B03DISSIMILARITY
            plan.append((band_idx, var, OUT_BASE / out_name(REGION_TAG, YEAR, PLATFORM, var, TILE), RESAMPLE_NUMERIC))
            band_idx += 1
    return plan

# ---- Execute split + reproject ----
def process_product(src_tif: Path, plan_func):
    if not src_tif or not src_tif.exists():
        print(f"• Missing: {src_tif} — skipping")
        return []
    created = []
    for band_idx, var, dst, resample in plan_func(src_tif):
        if dst.exists():
            print(f"• Skip (exists): {dst.name}")
            created.append(dst)
            continue
        tmp = dst.with_suffix(".tmp.tif")
        try:
            gdal_translate_band(src_tif, band_idx, tmp, NODATA)
            gdal_warp_reproject(tmp, dst, TARGET_EPSG, NODATA, resample=resample)
            created.append(dst)
        finally:
            try: tmp.unlink()
            except Exception: pass
    return created

# ---- Run for S2TC, BIO (per-date + TC), GLCM ----
built = []

if SRC_S2TC:
    print(f"[S2TC] {SRC_S2TC.name}")
    built += process_product(SRC_S2TC, plan_tc)
else:
    print("• No S2TC found — skipping")

if SRC_BIO:
    for bio_tif in SRC_BIO:
        print(f"[BIO] {bio_tif.name}")
        built += process_product(bio_tif, plan_bio)
else:
    print("• No BIO tifs found — skipping")

if SRC_GLCM:
    print(f"[GLCM] {SRC_GLCM.name}")
    built += process_product(SRC_GLCM, plan_glcm)
else:
    print("• No GLCM found — skipping")

# ---- Inventory CSV ----
rows = []
for p in sorted(set(built)):
    # StudyArea_YYYY0000_S2_<VAR>_<TILE>.tif
    parts = p.stem.split("_")
    var = parts[-2] if len(parts) >= 2 else "UNKNOWN"
    rows.append([REGION_TAG, YEAR, PLATFORM, TILE, var, str(p)])

with open(INV_CSV, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["study_area","year","platform","tile","variable","path"])
    w.writerows(rows)

print(f"\n✓ Dataset ready in {OUT_BASE}")
print(f"  • Singles written: {len(rows)}")
print(f"  • Inventory: {INV_CSV}")


== S2 Data Builder ==
• S2TC: ['Teruel_20220000_S2TC_T30TXK.tif']
• BIO_TC: ['Teruel_20220000_S2BIOTC_T30TXK.tif']
• BIO per-date: 6
• GLCM: ['Teruel_20220000_GLCM_T30TXK.tif']
[S2TC] Teruel_20220000_S2TC_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B01_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B02_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B03_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B04_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B05_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B06_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B07_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B08_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B8A_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B09_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B11_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_B12_T30TXK.tif
[BIO] Teruel_20220000_S2BIOTC_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_LAI_T30TXK.tif
• Skip (exists): Teruel_20220000_S2_FAPAR_T30TXK.tif
• Skip (exists): Teruel_202

KeyboardInterrupt: 

In [35]:
# Final cleanup: obey PUBLISH_MODE
# - Always clean intermediates under work/<PLOT>/<TILE>/
# - In release mode: prune deliverables to keep only DATASET/

WORK_ROOT = Path(r"A:\S2\work")
D_ROOT    = Path(r"A:\S2\Deliverables")

# Safety switches
DRY_RUN          = False   # True = preview only
ALLOW_IF_PARTIAL = False   # True = ignore missing must-haves

# ---- Deliverables root for this tile ----
D_TILE = DELIV
DATASET_DIR = D_TILE / "DATASET"

# ---- Must-have gate (release integrity) ----
must_have = {
    "DATASET_DIR": DATASET_DIR,
    "README":      DATASET_DIR / "README_bandmap.txt",
}

missing_must = [k for k, p in must_have.items() if not p.exists()]
if missing_must and not ALLOW_IF_PARTIAL:
    print("⚠️ Cleanup aborted: required release artifacts missing:")
    for k in missing_must:
        print(f"  - {k}: {must_have[k]}")
    print("Set ALLOW_IF_PARTIAL=True if you still want to proceed.")
    raise SystemExit(1)

# =====================================================================
# WORK cleanup (always)
# =====================================================================
work_tile = WORK_ROOT / PLOT / TILE

rm_work_dirs = [
    work_tile / "REF25",
    work_tile / "BIO25",
    work_tile / "SCL25",
    work_tile / "COMPOSITE25",
    work_tile / "GLCM25",
    work_tile / "DEBUG",
    work_tile / "BIO_TC",
    work_tile / "COLLOC",
    work_tile / "_graphs",
]

rm_work_files = list(work_tile.glob("*.tmp")) + list(work_tile.glob("*.log"))

# =====================================================================
# DELIVERABLE pruning (depends on PUBLISH_MODE)
# =====================================================================
KEEP_TILE_DIRS = {
    "dev":     {"S2TC", "BIO", "GLCM", "DATASET"},
    "release": {"DATASET"},
}[PUBLISH_MODE]

# Delete any directory in D_TILE that isn't in KEEP_TILE_DIRS
rm_deliv_dirs = [p for p in D_TILE.iterdir() if p.is_dir() and p.name not in KEEP_TILE_DIRS]

# Optional: also delete stray files at tile root (release)
rm_deliv_files = []
if PUBLISH_MODE == "release":
    rm_deliv_files = [p for p in D_TILE.iterdir() if p.is_file()]

# =====================================================================
# PRINT PLAN
# =====================================================================
print("\n== CLEANUP PLAN ==")
print(f"PUBLISH_MODE = {PUBLISH_MODE}")

print("\n[WORK] Will DELETE dirs:")
for d in rm_work_dirs:
    if d.exists(): print("  [DIR]", d)

print("[WORK] Will DELETE files:")
for f in rm_work_files:
    if f.exists(): print("  [FIL]", f)

print("\n[DELIV] Will DELETE dirs:")
for d in rm_deliv_dirs:
    if d.exists(): print("  [DIR]", d)

print("[DELIV] Will DELETE files:")
for f in rm_deliv_files:
    if f.exists(): print("  [FIL]", f)

print("\nMust-have check:")
for k, p in must_have.items():
    print(f"  {k:12s}: {'OK' if p.exists() else 'MISSING'}  -> {p}")

# =====================================================================
# EXECUTION
# =====================================================================
if DRY_RUN:
    print("\nDRY_RUN=True — no deletions performed.")
else:
    def safe_rmtree(path: Path, root: Path):
        path = path.resolve()
        root = root.resolve()
        if root not in path.parents and path != root:
            raise RuntimeError(f"Refusing to delete outside root={root}: {path}")
        shutil.rmtree(path, ignore_errors=True)

    def safe_unlink(path: Path, root: Path):
        path = path.resolve()
        root = root.resolve()
        if root not in path.parents and path != root:
            raise RuntimeError(f"Refusing to delete outside root={root}: {path}")
        try:
            path.unlink()
        except FileNotFoundError:
            pass

    # ---- WORK ----
    for d in rm_work_dirs:
        if d.exists():
            print("🧹 [WORK] rmtree:", d)
            safe_rmtree(d, WORK_ROOT)

    for f in rm_work_files:
        if f.exists():
            print("🧽 [WORK] unlink:", f)
            safe_unlink(f, WORK_ROOT)

    # ---- DELIVERABLES ----
    for d in rm_deliv_dirs:
        if d.exists():
            print("🧹 [DELIV] rmtree:", d)
            safe_rmtree(d, D_ROOT)

    for f in rm_deliv_files:
        if f.exists():
            print("🧽 [DELIV] unlink:", f)
            safe_unlink(f, D_ROOT)

    print("\n✅ Cleanup complete.")



== CLEANUP PLAN ==
PUBLISH_MODE = release

[WORK] Will DELETE dirs:
  [DIR] A:\S2\work\Teruel\T30TXK\REF25
  [DIR] A:\S2\work\Teruel\T30TXK\BIO25
  [DIR] A:\S2\work\Teruel\T30TXK\SCL25
  [DIR] A:\S2\work\Teruel\T30TXK\COMPOSITE25
  [DIR] A:\S2\work\Teruel\T30TXK\GLCM25
  [DIR] A:\S2\work\Teruel\T30TXK\BIO_TC
  [DIR] A:\S2\work\Teruel\T30TXK\COLLOC
  [DIR] A:\S2\work\Teruel\T30TXK\_graphs
[WORK] Will DELETE files:

[DELIV] Will DELETE dirs:
  [DIR] A:\S2\Deliverables\Teruel\T30TXK\S2TC
  [DIR] A:\S2\Deliverables\Teruel\T30TXK\BIO
  [DIR] A:\S2\Deliverables\Teruel\T30TXK\GLCM
  [DIR] A:\S2\Deliverables\Teruel\T30TXK\PCA

Must-have check:
  DATASET_DIR : OK  -> A:\S2\Deliverables\Teruel\T30TXK\DATASET
  README      : OK  -> A:\S2\Deliverables\Teruel\T30TXK\DATASET\README_bandmap.txt
🧹 [WORK] rmtree: A:\S2\work\Teruel\T30TXK\REF25
🧹 [WORK] rmtree: A:\S2\work\Teruel\T30TXK\BIO25
🧹 [WORK] rmtree: A:\S2\work\Teruel\T30TXK\SCL25
🧹 [WORK] rmtree: A:\S2\work\Teruel\T30TXK\COMPOSITE25
🧹 [WORK] r